In [25]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import math
import datetime as dt

In [2]:
# n1 = 북한 남쪽
# n2 = 북한 북쪽
# n3 = 블라디보스톡
# n4 = 일본 동쪽
# n5 = 일본 서쪽
# n6 = 중국 북쪽
# n7 = 중국 남쪽

In [3]:
def wind_direct(s):
    if s.value_counts().head(1).index == 99:
        return 99
    else:
        return s[s.values != 99].mean()

In [4]:
# 파일 목록 연도
year = []
for i in range(1996,2022):
    year.append(i)
    
# year

In [5]:
cities = pd.DataFrame()

for y in year:

    # 파일불러오기
    df = pd.read_csv('../data/러일중 1996~현재 기후(시간단위)/러일중 {}(시간).csv'.format(y), encoding='euckr')
    df_n = pd.read_csv('../data/북한 기후 시간단위/북한 {}(시간).csv'.format(y), encoding='euckr')

    # 변화량기압 drop
    df.drop('변화량 기압', axis=1, inplace=True)
    idx_r = df[df['지점명']=='상페테르부르크'].index
    df = df.drop(idx_r)
    idx_r = df[df['지점명']=='모스코바'].index
    df = df.drop(idx_r)

    # 북한 rename해주기
    df_n.rename(columns={'지점':'지점명'}, inplace=True)
    df_n.rename(columns={'풍향(36방위)':'풍향'}, inplace=True)


    # 일시 시간 버리기
    df['일시'] = df['일시'].apply(lambda x:x.split()[0])
    df_n['일시'] = df_n['일시'].apply(lambda x:x.split()[0])

    # 풍향 nan값 99로 바꾸기
    df['16 방위 풍향'].fillna(99,inplace=True)
    df['16 방위 풍향'] = df['16 방위 풍향'].apply(lambda x : 99 if x > 36 else x)

    df_n['풍향'].fillna(99,inplace=True)
    df_n['풍향'] = df_n['풍향'].apply(lambda x : 99 if x > 36 else x)

    # 방향마다 풍향 계산
    df['x_axis'] = df['16 방위 풍향'].apply(lambda x: math.sin(math.radians(x*10)) if 0<x<37 else x)
    df['y_axis'] = df['16 방위 풍향'].apply(lambda x: math.cos(math.radians(x*10)) if 0<x<37 else x)

    df_n['x_axis_n'] = df_n['풍향'].apply(lambda x: math.sin(math.radians(x*10)) if 0<x<37 else x)
    df_n['y_axis_n'] = df_n['풍향'].apply(lambda x: math.cos(math.radians(x*10)) if 0<x<37 else x)

    # 풍향 데이터프레임 만들기
    tdf = df.groupby(['지점명','일시'])['x_axis', 'y_axis'].agg(wind_direct).reset_index()
    tdf_n = df_n.groupby(['지점명','일시'])['x_axis_n', 'y_axis_n'].agg(wind_direct).reset_index()

    # 일일 풍향 만들기
    tmplst = []
    for ind in range(len(tdf)):
        if tdf.loc[ind,'x_axis'] == 99:
            tmplst.append(1000)
        else:
            tmplst.append(math.degrees(math.atan(tdf.loc[ind,'x_axis']/tdf.loc[ind,'y_axis'])))
    tdf['일일 풍향']=tmplst

    tmplst_n = []
    for ind_n in range(len(tdf_n)):
        if tdf_n.loc[ind_n,'x_axis_n'] == 99:
            tmplst_n.append(1000)
        else:
            tmplst_n.append(math.degrees(math.atan(tdf_n.loc[ind_n,'x_axis_n']/tdf_n.loc[ind_n,'y_axis_n'])))
    tdf_n['일일 풍향']=tmplst_n

    #
    tdf['일일 풍향'] = tdf['일일 풍향'].apply(lambda x: round(x+360,ndigits=-1) if x<0 else round(x,ndigits=-1)).fillna(0)
    tdf_n['일일 풍향'] = tdf_n['일일 풍향'].apply(lambda x: round(x+360,ndigits=-1) if x<0 else round(x,ndigits=-1)).fillna(0)

    # 풍향을 제외한 내용들 일일로 묶어주기
    df_tt = df.groupby(['지점명', '일시']).agg({'기온':['mean','max','min'], '습도':'mean', 
                                            '풍속':'mean', '강수량':'sum','해면기압':'mean', '이슬점 온도':'mean'})

    df_tt.columns = ['평균기온','최고기온','최저기온','습도','풍속','강수량','해면기압','이슬점 온도']
    df_city = df_tt.reset_index()

    df_tt_n = df_n.groupby(['지점명', '일시']).agg({'기온(°C)':['mean','max','min'], '상대습도(%)':'mean', 
                                            '풍속(m/s)':'mean', '12시간강수량(mm)':'sum','해면기압(hPa)':'mean',
                                            '이슬점온도(°C)':'mean'})

    df_tt_n.columns = ['평균기온','최고기온','최저기온','습도','풍속','강수량','해면기압','이슬점 온도']
    df_city_n = df_tt_n.reset_index()

    # 풍향데이터와 나머지 데이터 concat
    df_city = pd.concat([df_city,tdf['일일 풍향']],axis=1)
    df_city_n = pd.concat([df_city_n,tdf_n['일일 풍향']],axis=1)


    # 컬럼이름 순서 바꾸기
    df_city = df_city[['지점명', '일시', '평균기온', '최고기온', '최저기온', '습도', '일일 풍향','풍속',  '강수량', '해면기압', 
           '이슬점 온도']]
    df_city.rename({'일일 풍향':'풍향'},axis = 1,inplace=True)

    df_city_n = df_city_n[['지점명', '일시', '평균기온', '최고기온', '최저기온', '습도', '일일 풍향','풍속',  '강수량', '해면기압',
           '이슬점 온도']]
    df_city_n.rename({'일일 풍향':'풍향'},axis = 1,inplace=True)

    # 군집번호 정하기
    j_li = ['도쿄','오사카','나고야','삿포로','센다이','니가타','히로시마','다카마쓰','후쿠오카','가고시마','오키나와','난징','베이징','상하이','선양','창춘','텐진','다롄','지난','블라디보스톡']
    j_li_n = ['n4','n5','n5','n4','n4','n4','n5','n5','n5','n5','n5','n7','n6','n7','n6','n6','n6','n6','n7','n3']

    no = [5, 22, 16, 20, 14, 39, 46, 31, 41, 25, 3, 8]
    li_c=[]
    for i in df_city_n['지점명']:
        if i in no:
            li_c.append('n2')
        else:
            li_c.append('n1')

    # 군집번호 넣어주기
    df_city['군집'] = df_city['지점명'].apply(lambda x : j_li_n[j_li.index(x)] if x in j_li else np.nan)
    city = df_city[df_city['군집'].isna()==False]

    df_city_n['군집'] = li_c



    # 러일중 + 북한
    city_sum = pd.concat([df_city_n,city])
    
    # 연도별 모두 합치기
    cities = pd.concat([cities, city_sum])

# 인덱스 reset
cities = cities.reset_index()

# 파일 저장
cities.to_csv('주변국_기후.csv')

KeyboardInterrupt: 

In [8]:
df_w = pd.read_csv('../data/주변국_기후.csv', index_col=0)
df_w.drop('index', axis=1, inplace=True)
df_w

,지점명,일시,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도,군집
0,3,1996-01-01,-6.542857,-1.5,-9.8,47.000000,330.0,2.571429,0.0,1018.785714,-16.400000,n2
1,3,1996-01-02,-4.687500,1.5,-10.4,75.500000,360.0,0.500000,0.0,1014.662500,-8.625000,n2
2,3,1996-01-03,-7.400000,-1.1,-11.6,57.625000,320.0,8.250000,0.0,1010.962500,-14.600000,n2
3,3,1996-01-04,-8.862500,-5.0,-12.0,48.375000,330.0,2.375000,0.0,1020.400000,-18.237500,n2
4,3,1996-01-05,-6.300000,-0.8,-10.5,79.625000,20.0,0.500000,0.0,1021.712500,-9.425000,n2
...,...,...,...,...,...,...,...,...,...,...,...,...
420345,히로시마,2021-12-27,3.295833,5.3,0.6,60.487500,320.0,4.500000,0.5,1026.608333,-3.920833,n5
420346,히로시마,2021-12-28,5.887500,9.5,3.2,62.079167,350.0,4.416667,-99.8,1026.020833,-1.250000,n5
420347,히로시마,2021-12-29,7.162500,9.2,4.3,72.129167,80.0,4.583333,-98.8,1018.200000,2.329167,n5
420348,히로시마,2021-12-30,4.291667,8.8,0.4,59.554167,280.0,6.625000,-97.8,1020.458333,-3.316667,n5


In [9]:
north = ['평양', '해주', '원산', '함흥', '강계', '신의주', '청진', 
            '혜산', '남포', '사리원', '개성', '용연', '안주', '삼지연', '선봉', '신포', '수풍','양덕', '신계', '풍산', '구성',
          '장전','희천','김책','평강','장진','중강']

In [10]:
north_n = ['58', '69', '55', '41', '20', '35', '8', '16', '60', '65', '70', '68', '50', '5', '3', '46', '28', '52', '67', '22', '37', '61', '39', '25', '75', '31', '14']

In [11]:
df_w['지점명'] = df_w['지점명'].apply(lambda x : north[north_n.index(x)] if x in north_n else x)

In [12]:
# df_w.drop('도시명', axis=1, inplace=True)

In [13]:
df_w.rename({'지점명':'도시명'},axis = 1,inplace=True)

In [14]:
df_w

,도시명,일시,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도,군집
0,선봉,1996-01-01,-6.542857,-1.5,-9.8,47.000000,330.0,2.571429,0.0,1018.785714,-16.400000,n2
1,선봉,1996-01-02,-4.687500,1.5,-10.4,75.500000,360.0,0.500000,0.0,1014.662500,-8.625000,n2
2,선봉,1996-01-03,-7.400000,-1.1,-11.6,57.625000,320.0,8.250000,0.0,1010.962500,-14.600000,n2
3,선봉,1996-01-04,-8.862500,-5.0,-12.0,48.375000,330.0,2.375000,0.0,1020.400000,-18.237500,n2
4,선봉,1996-01-05,-6.300000,-0.8,-10.5,79.625000,20.0,0.500000,0.0,1021.712500,-9.425000,n2
...,...,...,...,...,...,...,...,...,...,...,...,...
420345,히로시마,2021-12-27,3.295833,5.3,0.6,60.487500,320.0,4.500000,0.5,1026.608333,-3.920833,n5
420346,히로시마,2021-12-28,5.887500,9.5,3.2,62.079167,350.0,4.416667,-99.8,1026.020833,-1.250000,n5
420347,히로시마,2021-12-29,7.162500,9.2,4.3,72.129167,80.0,4.583333,-98.8,1018.200000,2.329167,n5
420348,히로시마,2021-12-30,4.291667,8.8,0.4,59.554167,280.0,6.625000,-97.8,1020.458333,-3.316667,n5


In [15]:
df_w.to_csv('../data/주변국_기후_도시명.csv')

In [16]:
df_w = pd.read_csv('../data/주변국_기후_도시명.csv', index_col=0)

In [17]:
df_w

,도시명,일시,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도,군집
0,선봉,1996-01-01,-6.542857,-1.5,-9.8,47.000000,330.0,2.571429,0.0,1018.785714,-16.400000,n2
1,선봉,1996-01-02,-4.687500,1.5,-10.4,75.500000,360.0,0.500000,0.0,1014.662500,-8.625000,n2
2,선봉,1996-01-03,-7.400000,-1.1,-11.6,57.625000,320.0,8.250000,0.0,1010.962500,-14.600000,n2
3,선봉,1996-01-04,-8.862500,-5.0,-12.0,48.375000,330.0,2.375000,0.0,1020.400000,-18.237500,n2
4,선봉,1996-01-05,-6.300000,-0.8,-10.5,79.625000,20.0,0.500000,0.0,1021.712500,-9.425000,n2
...,...,...,...,...,...,...,...,...,...,...,...,...
420345,히로시마,2021-12-27,3.295833,5.3,0.6,60.487500,320.0,4.500000,0.5,1026.608333,-3.920833,n5
420346,히로시마,2021-12-28,5.887500,9.5,3.2,62.079167,350.0,4.416667,-99.8,1026.020833,-1.250000,n5
420347,히로시마,2021-12-29,7.162500,9.2,4.3,72.129167,80.0,4.583333,-98.8,1018.200000,2.329167,n5
420348,히로시마,2021-12-30,4.291667,8.8,0.4,59.554167,280.0,6.625000,-97.8,1020.458333,-3.316667,n5


In [18]:
df_w.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420350 entries, 0 to 420349
Data columns (total 12 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   도시명     420350 non-null  object 
 1   일시      420350 non-null  object 
 2   평균기온    420313 non-null  float64
 3   최고기온    420313 non-null  float64
 4   최저기온    420313 non-null  float64
 5   습도      419923 non-null  float64
 6   풍향      420350 non-null  float64
 7   풍속      420322 non-null  float64
 8   강수량     420350 non-null  float64
 9   해면기압    395035 non-null  float64
 10  이슬점 온도  420321 non-null  float64
 11  군집      420350 non-null  object 
dtypes: float64(9), object(3)
memory usage: 41.7+ MB


In [19]:
df_w.isna().sum()

도시명           0
일시            0
평균기온         37
최고기온         37
최저기온         37
습도          427
풍향            0
풍속           28
강수량           0
해면기압      25315
이슬점 온도       29
군집            0
dtype: int64

In [ ]:
# 평온, 고온, 저온, 이슬점 - 보간
# 습도, 풍속, 해면기압 - 지역평균

In [21]:
df_w['도시명'].unique()

array(['선봉', '삼지연', '청진', '중강', '혜산', '강계', '풍산', '김책', '수풍', '장진', '신의주',
       '구성', '희천', '함흥', '신포', '안주', '양덕', '원산', '평양', '남포', '장전', '사리원',
       '신계', '용연', '해주', '개성', '평강', '가고시마', '나고야', '난징', '다롄', '다카마쓰',
       '도쿄', '베이징', '블라디보스톡', '삿포로', '상하이', '선양', '센다이', '오사카', '지난',
       '창춘', '텐진', '후쿠오카', '히로시마'], dtype=object)

In [116]:
dellst = '히로시마,양덕,수풍,삼지연,안주,선봉'.split(',')
for tmp in '삼지연,중강,장진,평강,김책,희천,장전,구성,풍산,신계,신포,텐진,창춘,다롄,지난,난징,베이징,선양,상하이'.split(','):
    if tmp in north:
        dellst.append(tmp)
dellst
    

['히로시마',
 '양덕',
 '수풍',
 '삼지연',
 '안주',
 '선봉',
 '삼지연',
 '중강',
 '장진',
 '평강',
 '김책',
 '희천',
 '장전',
 '구성',
 '풍산',
 '신계',
 '신포']

In [117]:
lst = []
for tmp in df_w['도시명'].unique():
    if not tmp in dellst:
        lst.append(tmp)
lst

['청진',
 '혜산',
 '강계',
 '신의주',
 '함흥',
 '원산',
 '평양',
 '남포',
 '사리원',
 '용연',
 '해주',
 '개성',
 '가고시마',
 '나고야',
 '난징',
 '다롄',
 '다카마쓰',
 '도쿄',
 '베이징',
 '블라디보스톡',
 '삿포로',
 '상하이',
 '선양',
 '센다이',
 '오사카',
 '지난',
 '창춘',
 '텐진',
 '후쿠오카']

In [120]:
global_df = pd.DataFrame()
for city in lst:
    if df_w[df_w['도시명']==city]['군집'].unique()[0] == 'n6' \
        or df_w[df_w['도시명']==city]['군집'].unique()[0] == 'n7':
        tmp = pd.DataFrame({'일시':np.arange(dt.datetime(1996,1,1),dt.datetime(2021,9,1),dt.timedelta(days=1))})
    else:
        tmp = pd.DataFrame({'일시':np.arange(dt.datetime(1996,1,1),dt.datetime(2021,9,1),dt.timedelta(days=1))})
    tmp = pd.merge(tmp.astype('str'), df_w[df_w['도시명']==city].drop('도시명',axis=1),how='left',on='일시')
    tmp[['평균기온','최고기온','최저기온','습도','해면기압','이슬점 온도']] = tmp[['평균기온','최고기온','최저기온','습도','해면기압','이슬점 온도']].interpolate()
    tmp[['풍향','풍속']] = tmp[['풍향','풍속']].fillna(tmp[['풍향','풍속']].mean())
    tmp['강수량'].fillna(0,inplace=True)
    tmp['군집'].fillna(tmp['군집'].unique()[0],inplace=True)
    tmp['도시명'] = city
    ttmp=tmp[['평균기온', '최고기온', '최저기온', '습도', '풍향', '풍속', '강수량', '해면기압','이슬점 온도']]
    low, high = np.percentile(ttmp,[25,75],axis=0)
    tmp[['평균기온', '최고기온', '최저기온', '습도', '풍향', '풍속', '강수량', '해면기압','이슬점 온도']] = ttmp[(ttmp>=low-(high-low)*2.5)&(ttmp<=high+(high-low)*2.5)].fillna(ttmp.mean())
    tmp = tmp[['도시명', '일시', '평균기온', '최고기온', '최저기온', '습도', '풍향', '풍속', '강수량', '해면기압','이슬점 온도', '군집']]
    global_df = pd.concat([global_df,tmp])
global_df.to_csv('../data/preprocessed_주변국 기상데이터_199601to202108.csv')
global_df


,도시명,일시,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도,군집
0,청진,1996-01-01,-5.528571,-0.1,-9.9,53.571429,330.0,0.571429,0.0,1018.685714,-13.657143,n2
1,청진,1996-01-02,-3.825000,1.3,-8.2,72.875000,80.0,0.500000,0.0,1014.037500,-8.025000,n2
2,청진,1996-01-03,-6.912500,-1.4,-11.7,48.375000,330.0,3.000000,0.0,1010.537500,-16.100000,n2
3,청진,1996-01-04,-9.475000,-3.4,-19.2,56.500000,320.0,0.250000,0.0,1020.800000,-16.625000,n2
4,청진,1996-01-05,-4.487500,0.5,-9.9,71.750000,290.0,0.375000,0.0,1021.800000,-8.837500,n2
...,...,...,...,...,...,...,...,...,...,...,...,...
9370,후쿠오카,2021-08-27,29.175000,32.8,25.6,70.495833,30.0,5.125000,0.0,1014.391667,23.162500,n5
9371,후쿠오카,2021-08-28,28.641667,32.3,25.1,72.362500,40.0,4.958333,0.0,1014.325000,23.000000,n5
9372,후쿠오카,2021-08-29,28.908333,32.4,25.5,72.687500,290.0,5.041667,0.0,1014.754167,23.375000,n5
9373,후쿠오카,2021-08-30,28.904167,32.5,25.8,75.795833,310.0,4.541667,0.0,1015.587500,24.112500,n5


In [100]:
df_w.columns

Index(['도시명', '일시', '평균기온', '최고기온', '최저기온', '습도', '풍향', '풍속', '강수량', '해면기압',
       '이슬점 온도', '군집'],
      dtype='object')

In [101]:
# 버림
# 중강, 장진, 평강, 김책, 희천, 장전, 구성, 풍산, 신계, 신포, 텐진, 창춘, 다롄, 지난, 난징, 베이징, 선양, 상하이, 

In [102]:
# 후보
# 히로시마, 양덕, 수풍, 삼지연, 안주, 선봉

In [119]:
global_df.isna().sum()

도시명       0
일시        0
평균기온      0
최고기온      0
최저기온      0
습도        0
풍향        0
풍속        0
강수량       0
해면기압      0
이슬점 온도    0
군집        0
dtype: int64

In [104]:
global_df[global_df['해면기압'].isna()]

,도시명,일시,평균기온,최고기온,최저기온,습도,풍향,풍속,강수량,해면기압,이슬점 온도,군집


In [108]:
city = '상해'
tmp = pd.DataFrame({'일시':np.arange(dt.datetime(1996,1,1),dt.datetime(2022,1,1),dt.timedelta(days=1))})
tmp = pd.merge(tmp.astype('str'), df_w[df_w['도시명']==city].drop('도시명',axis=1),how='left',on='일시')
tmp[tmp['평균기온'].isna()]['일시'].values

array(['1996-01-01', '1996-01-02', '1996-01-03', ..., '2021-12-29',
       '2021-12-30', '2021-12-31'], dtype=object)

In [87]:
global_df[global_df['군집'] == 'n2']['도시명'].unique()

array(['선봉', '청진', '혜산', '강계', '함흥'], dtype=object)

In [ ]:
'수풍', '신의주', '안주', '양덕', '원산', '평양', '남포', '사리원', '용연', '해주', '개성','선봉', '청진', '혜산', '강계', '함흥'

In [ ]:
# 히로시마, 양덕, 수풍, 삼지연, 안주, 선봉

In [ ]:
제낄거 수풍, 안주 선봉 삼지연

In [ ]:
양덕 - 원산